In [ ]:
from types import SimpleNamespace
import numpy as np
import deepdish as dd
import os
import holoviews as hv
hv.notebook_extension()
%opts Scatter (color='r')

# No disorder

In [ ]:
path = 'data_L_matsfreqs_a_T/'
files = [f for f in os.listdir(path) if f.endswith('.h5')]

fs = {f: dd.io.load(path + f) for f in files}

hm = {}
for fname, vals in fs.items():
    key = (vals['wire_params']['a'], vals['wire_params']['L'], vals['matsfreqs'], vals['T'])
    I_cs = vals['I_cs']
    val = hv.Curve((vals['B_xs'], I_cs), kdims=[hv.Dimension('$B$', unit='T')], vdims=['$I$'])
    hm[key] = val

hv.HoloMap(hm, kdims=[hv.Dimension('$a$', unit='nm'),
                      hv.Dimension('$L$', unit='nm'),
                      'matfreqs',
                      hv.Dimension('$T$', unit='K')])

In [ ]:
len(files)

# Disorder

In [ ]:
file = dd.io.load('data/L_600-matfreqs_1-a_10-T_0.06_disorder.h5')

In [ ]:
file.keys()

In [ ]:
p = file['p']
T = file['T']
matsfreqs = file['matsfreqs']
constants = file['constants']
Bs = file['B_xs']
phases = file['phases']
wire_params = file['wire_params']
I_c = file['I_cs']
disorders = file['disorders']
salts = file['salts']

In [ ]:
curves = {}
for i, realisation in enumerate(I_c):
    for disorder, I_cs in zip(disorders, realisation):
        curves[(i, disorder)] = hv.Curve((Bs, I_cs), kdims=[hv.Dimension('$B$', unit='$T$')], vdims=['$I_c$'])
hm = hv.HoloMap(curves, kdims=['salt', 'disorder'])

In [ ]:
hm

In [ ]:
collapsed = hm.collapse('salt', function=np.mean, spreadfn=np.std)
disorder = collapsed * collapsed.table().to.spread(['$B$', '$I_c$'], '$I_c$_std')
disorder

In [ ]:
%%output size=200 fig='svg'
hv.GridSpace(disorder)

# Mean free path $g \sim N_{ch} / (1 + L / \lambda_{MFP})$
Check with Carlo's review paper on random matrix theory

In [ ]:
mfp = dd.io.load('data/mfp.h5')

In [ ]:
import xarray as xr
disorders = mfp['disorders']
conductances = xr.DataArray(mfp['conductances'], dims=['length', 'salts', 'disorder'])
Ls = np.array(mfp['Ls']) - 20
num_propagating = np.unique(mfp['num_propagating'])
assert len(num_propagating) == 1

In [ ]:
hm_dict = {}
for L, conductance in zip(Ls, conductances):
    conductance_mean = conductance.mean(dim='salts')
    curve = hv.Curve((disorders, conductance_mean), kdims=['disorder'], vdims=['conductance'])
    spread = hv.Spread((disorders, conductance_mean, conductance.std(dim='salts')))
    scatter = hv.Scatter((disorders, conductance_mean))
    hm_dict[L] = curve * spread * scatter
hv.HoloMap(hm_dict, kdims=['L'])

In [ ]:
hm_dict = {}
for disorder, resistance in zip(disorders, (1 / conductances.T)):
    resistance_mean = resistance.mean(dim='salts')
    curve = hv.Curve((Ls, resistance_mean), kdims=['L'], vdims=['resistance'])
    spread = hv.Spread((Ls, resistance_mean, resistance.std(dim='salts')))
    scatter = hv.Scatter((Ls, resistance_mean))
    hm_dict[disorder] = curve * spread * scatter
hm = hv.HoloMap(hm_dict, kdims=['disorder'])
hm

In [ ]:
from functools import partial

def R(l_mfp, L, N_ch=12):
    return (1 + L / l_mfp) / N_ch

R_ = partial(R, N_ch=12)

fits = np.squeeze([np.polyfit(Ls, resistance, deg=1)[0] for resistance in (1/conductances).mean('salts').transpose('disorder', 'length')])
l_mfps = 1 / (12 * fits)

## Fit and data

In [ ]:
dims = dict(kdims=['L'], vdims=['resistance'])
hv.HoloMap({disorder: hv.Curve((Ls, R_(l_mfp, Ls)), **dims) for disorder, l_mfp in zip(disorders, l_mfps)}, 
           kdims=['disorder']) * hm

In [ ]:
for disorder, l_mfp in zip(disorders, l_mfps):
    print("Disorder strength: {} meV and mean free path: {:.5} nm".format(disorder, l_mfp))